# Gemini Notebook  
This notebook demonstrates the usage of Gemini APIs integrated with Javelin.


In [4]:
import json
import os
from openai import OpenAI
from javelin_sdk import (
    JavelinClient,
    JavelinConfig,
)

# -------------------------------
# Gemini API Example
# -------------------------------
print("Initializing Gemini client...")

# Retrieve the Gemini API key from the environment variable
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set!")
print("Gemini API Key loaded successfully.")
# Create an OpenAI client configured for Gemini with the appropriate base URL
openai_client = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Initialize the Javelin Client (using the same javelin_api_key as before)
javelin_api_key = os.getenv('JAVELIN_API_KEY')
config = JavelinConfig(
    base_url="https://api-dev.javelin.live",
    # Uncomment the following line to use a local server:
    # base_url="http://localhost:8000",
    javelin_api_key=javelin_api_key,
)
client = JavelinClient(config)

# Register the Gemini client with Javelin
client.register_gemini(openai_client, route_name="testing")

print("Gemini: 1 - Chat completions")
response = openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain to me how AI works"}
    ]
)
print(response.model_dump_json(indent=2))

print("Gemini: 2 - Streaming")
response = openai_client.chat.completions.create(
  model="gemini-1.5-flash",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ],
  stream=True
)
for chunk in response:
    # Debug print: output each streaming chunk's delta
    print(chunk.choices[0].delta)

print("Gemini: 3 - Function calling")
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "Get the weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. Chicago, IL",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    }
  }
]
messages = [{"role": "user", "content": "What's the weather like in Chicago today?"}]
response = openai_client.chat.completions.create(
  model="gemini-1.5-flash",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)
print(response.model_dump_json(indent=2))

# --- Gemini Image Understanding Example (Commented Out) ---
'''
print("Gemini: 4 - Image understanding")
import base64

# Function to encode the image in base64 format
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Convert an image to a base64 string
base64_image = encode_image("Path/to/agi/image.jpeg")

response = client.chat.completions.create(
  model="gemini-1.5-flash",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What is in this image?"},
        {"type": "image_url", "image_url": {"url":  f"data:image/jpeg;base64,{base64_image}"}},
      ],
    }
  ],
)
print(response.model_dump_json(indent=2))
'''

print("Gemini: 5 - Structured output")
from pydantic import BaseModel
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = openai_client.beta.chat.completions.parse(
    model="gemini-1.5-flash",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "John and Susan are going to an AI conference on Friday."},
    ],
    response_format=CalendarEvent,
)
print(completion.model_dump_json(indent=2))

print("Gemini: 6 - Embeddings")
response = openai_client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-004"
)
print(response.model_dump_json(indent=2))

# Prints two blank lines for clarity
print("\n\n")


Initializing Gemini client...


ValueError: GEMINI_API_KEY is not set!